# 동작구 Crawling1 : 대한민국 구석구석에서 한국 역사 여행지 불러오기

한국관광공사에서 여행정보를 제공하는 페이지인 '대한민국 구석구석'에서 전국의 여행지 목록을 열람할 수 있는데, 그 중 "역사"태그로 역사관련 문화유적지의 목록을 찾을 수 있다. 아래는 역사 관광지의 목록을 크롤링하는 코드이다.

In [1]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np

### Selenium과 BeautifulSoup로 parsing하기

In [2]:
header = {'User-Agent': ''}
d = webdriver.Chrome('C:/Users/user/Downloads/chromedriver') #웹앱을 테스트하는데 이용하는 프레임워크, webdriver라는 API를 통해 운영체제에 설치된 브라우저 제어 
d.implicitly_wait(3)

#페이지 불러오기
d.get('https://korean.visitkorea.or.kr/main/main.do') #한국관광공사 대한민국구석구석 메인사이트
d.find_element_by_xpath('/html/body/div[7]/div/button').click() # 코로나 바이러스 팝업 '닫기'클릭
d.find_element_by_xpath('/html/body/div[5]/ul/li[2]/a/em').click() # All여행지 항목 클릭
d.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[10]/a').click() # 역사 태그 클릭
#d.get("http://www.melon.com/chart/search/index.htm")
#d.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a').click() #멜론의 차트파인더에서 주간차트 선택
#https://korean.visitkorea.or.kr/main/main.do
req = requests.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%97%AD%EC%82%AC&choiceTagId=d3fd4d9f-fbd4-430f-b5d5-291b4d9920be')
html = req.text #html소스 가져오기, python의 문자열 객체를 반환
soup = BeautifulSoup(html, "html.parser") #html코드를 python이 이해하는 객체 구조로 변환하는 parsing으로 의미있는 정보 추출 가능

### 관광지 목록 불러오기

In [3]:
click_idx=0
f=open('placeList.txt','w',-1,'UTF-8') #역사 관광지의 이름 목록을 크롤링해 txt문서와 place 리스트에 저장한다
#f1=open('imgList.txt','w',-1,'UTF-8')

place = [] #장소명 저장하는 리스트
#img = [] #이미지 링크 저장하는 리스트

for j in range(1, 384): #마지막 페이지수는 바뀔 수 있음, 20.4.25기준 383페이지
    
    
    for i in range(1,11): #한페이지 당 열개의 관광지 목록 
    
            place_xpath = '//*[@id="contents"]/div[2]/div[1]/ul/li['+str(i)+']/div[2]/div/a' #관광지 이름 path
            #img_xpath = '//*[@id="contents"]/div[2]/div[1]/ul/li['+str(i)+']/div[1]/a/img' #관광지 사진 링크 path
            p = ""
            #img = ""
            try:
                p = d.find_element_by_xpath(place_xpath)
                #img = d.find_element_by_xpath(img_xpath)
                place.append(p.text)
                
                f.write(p.text+'\n')
                #f1.write(img.text+'\n')
                img.append(img)

            except: pass 
            
    #다음버튼 클릭하기(1,2,3,4,5 클릭하고 다음버튼), 다음페이지로 넘어가기
    try:
        if j<=5: #1~5페이지까지는 <<,<버튼이 존재하지 않아서 클릭할 인덱스가 다르다
            page_box=d.find_element_by_class_name('page_box')
            page_num=page_box.find_elements_by_tag_name('a')
            page_num[j].click()
            sleep(5)
        else: #6페이지~383페이지
            page_box=d.find_element_by_class_name('page_box')
            page_num=page_box.find_elements_by_tag_name('a')
            page_num[3+click_idx].click()
            click_idx+=1
            sleep(5)
            if click_idx==5: click_idx=0
                
    except: pass #except안해서 Index Error난것이니 신경쓰지마셈
    

f.close()        

배열 place에 '대한민국 구석구석' 의 관광지 목록 중 #역사 인 관광지들의 이름을 모두 불러와 text파일에 저장하였다.

2020.4.25기준 3830개의 관광지가 업데이트되어있음

->분석의 정확성을 높이고 필요없는 데이터는 제거하기위해 인기순으로 정렬 후 1000개의 장소만을 크롤링 하였음

In [4]:
print(len(place)) 

3830
